In [3]:
from scrape_league_data import create_underdog_df_dict
from pull_bearer_token import pull_bearer_token
import getpass
import pandas as pd

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

### Variables to change ###
chromedriver_path = '/usr/bin/chromedriver'
username = input('Enter Underdog username: ')
password = getpass.getpass()

### Keep as is ###
url = "https://underdogfantasy.com/lobby"    
bearer_token = pull_bearer_token(url, chromedriver_path, username, password)

### Pull all major UD data elements ###
# underdog_data = create_underdog_df_dict(bearer_token, sleep_time=5)

In [8]:
underdog_data['df_drafts'].to_csv('df_drafts.csv', index=False)
underdog_data['df_league_info'].to_csv('df_league_info.csv', index=False)